# 🚀 VectorBT Backtesting - Cryptocurrency Trading Bot

This notebook demonstrates the **VectorBT backtesting engine** replacing the manual candle-by-candle simulation approach.

## Features:
- ✅ Vectorized signal generation from XGBoost predictions
- ✅ High-performance backtesting with realistic commissions/slippage
- ✅ Comprehensive performance metrics (Sharpe, drawdown, win rate, etc.)
- ✅ Interactive visualizations (equity curve, trades, drawdown)
- ✅ Trade-by-trade analysis
- ✅ Export results to CSV

## Advantages over Manual Simulation:
- **10-100x faster** execution
- More realistic trade execution
- Professional-grade metrics
- Better visualization capabilities
- Easier parameter optimization

## 📦 Setup and Imports

In [1]:
# Add src to path
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"✅ Project root: {project_root}")
print(f"✅ Added to path: {src_path}")

# Core imports
import pandas as pd
import vectorbt as vbt

# Trading bot imports
from src.training.model_trainer import SimpleModelTrainer
from src.backtesting.vectorbt_engine import VectorBTBacktester

print("✅ All imports successful!")

✅ Project root: d:\Dev\trading-bot
✅ Added to path: d:\Dev\trading-bot\src
✅ All imports successful!
✅ All imports successful!


## 🎯 Load Trained Model

In [2]:
print("📁 Loading trained model from file...")
print("=" * 60)

# Create trainer instance
trainer = SimpleModelTrainer()

# Load model
success = trainer.load_model()

if not success or not trainer.is_trained:
    print("❌ Failed to load model!")
    print("   Please run the training cell in Simple_Model_Debug.ipynb first")
    raise ValueError("No trained model available")

# Display model info
info = trainer.get_model_info()
print("✅ Model loaded successfully!")
print(f"\n📊 Model Information:")
print(f"   Type: {info['model_type']}")
print(f"   Accuracy: {info['accuracy']:.2%}")
print(f"   Features: {info['features']}")
print(f"   Training Date: {info.get('training_date', 'Unknown')}")

print("\n" + "=" * 60)

📁 Loading trained model from file...
📁 Loading model from: src/models/simple_trading_model.joblib
✅ Model loaded: XGBoost-GPU (Accuracy: 98.2%)
✅ Model loaded successfully!

📊 Model Information:
   Type: XGBoost-GPU
   Accuracy: 98.18%
   Features: 60
   Training Date: Unknown



## 📊 Configure Backtest Parameters

In [3]:
# Backtest Configuration
SYMBOL = 'BTCUSDT'
DATA_FOLDER = 'data_test'
MAX_CANDLES = 1000  # Use more candles for realistic backtest (was 250 in simulation)

# Trading Parameters
BUY_THRESHOLD = 0.10   # 10% minimum confidence for BUY
SELL_THRESHOLD = 0.10  # 10% minimum confidence for SELL

# VectorBT Parameters
INITIAL_CASH = 100000.0  # Starting capital: $10,000
COMMISSION = 0.001      # 0.1% commission per trade (typical crypto exchange)
SLIPPAGE = 0.0005       # 0.05% slippage
FREQ = '15T'            # 15-minute candles

print("⚙️  BACKTEST CONFIGURATION")
print("=" * 60)
print(f"Symbol:           {SYMBOL}")
print(f"Data Folder:      {DATA_FOLDER}")
print(f"Max Candles:      {MAX_CANDLES}")
print(f"\n📈 Trading Thresholds:")
print(f"BUY Threshold:    {BUY_THRESHOLD:.1%}")
print(f"SELL Threshold:   {SELL_THRESHOLD:.1%}")
print(f"\n💰 VectorBT Settings:")
print(f"Initial Cash:     ${INITIAL_CASH:,.2f}")
print(f"Commission:       {COMMISSION:.2%}")
print(f"Slippage:         {SLIPPAGE:.2%}")
print(f"Frequency:        {FREQ}")
print("=" * 60)

⚙️  BACKTEST CONFIGURATION
Symbol:           BTCUSDT
Data Folder:      data_test
Max Candles:      1000

📈 Trading Thresholds:
BUY Threshold:    10.0%
SELL Threshold:   10.0%

💰 VectorBT Settings:
Initial Cash:     $100,000.00
Commission:       0.10%
Slippage:         0.05%
Frequency:        15T


## 📥 Load Test Data

In [4]:
from src.prediction.predictor import SimpleModelPredictor
import json

print("📥 Loading test data...")
print("=" * 60)

# Create predictor to access data loading utilities
predictor = SimpleModelPredictor(trainer)

# Get data files for symbol
test_files = predictor._get_symbol_files(SYMBOL, DATA_FOLDER)

if not test_files:
    raise ValueError(f"No data files found for {SYMBOL} in {DATA_FOLDER}")

print(f"✅ Found {len(test_files)} timeframe files:")
for tf, path in test_files.items():
    print(f"   {tf:>3s}: {path}")

# Load 15m data (primary timeframe)
if '15m' not in test_files:
    raise ValueError(f"No 15m data found for {SYMBOL}")

test_data = predictor._load_json_data(test_files['15m'])

if test_data is None:
    raise ValueError("Failed to load test data")

print(f"\n✅ Loaded {len(test_data)} total candles")
print(f"📅 Date Range: {test_data['datetime'].min()} to {test_data['datetime'].max()}")

# Get last N candles for backtest
if len(test_data) > MAX_CANDLES:
    test_data = test_data.tail(MAX_CANDLES).reset_index(drop=True)
    print(f"📊 Using last {MAX_CANDLES} candles for backtest")
else:
    print(f"📊 Using all {len(test_data)} candles")

print(f"📍 Backtest period: {test_data['datetime'].min()} to {test_data['datetime'].max()}")

# Set datetime as index (required for VectorBT)
test_data_indexed = test_data.set_index('datetime')

print("\n✅ Data preparation complete!")
print("=" * 60)

📥 Loading test data...
✅ Found 5 timeframe files:
   15m: data_test/BTCUSDT-15m.json
    1h: data_test/BTCUSDT-1h.json
     D: data_test/BTCUSDT-D.json
     W: data_test/BTCUSDT-W.json
     M: data_test/BTCUSDT-M.json

✅ Loaded 87891 total candles
📅 Date Range: 2023-04-03 13:45:00 to 2025-10-05 02:15:00
📊 Using last 1000 candles for backtest
📍 Backtest period: 2025-09-24 16:30:00 to 2025-10-05 02:15:00

✅ Data preparation complete!

✅ Loaded 87891 total candles
📅 Date Range: 2023-04-03 13:45:00 to 2025-10-05 02:15:00
📊 Using last 1000 candles for backtest
📍 Backtest period: 2025-09-24 16:30:00 to 2025-10-05 02:15:00

✅ Data preparation complete!


## 🎯 Initialize VectorBT Backtester

In [5]:
print("🚀 Initializing VectorBT Backtester...")
print("=" * 60)

# Create VectorBT backtester instance
backtester = VectorBTBacktester(
    trainer=trainer,
    initial_cash=INITIAL_CASH,
    commission=COMMISSION,
    slippage=SLIPPAGE
)

print("✅ VectorBT Backtester initialized!")
print(f"   Initial Cash:  ${backtester.initial_cash:,.2f}")
print(f"   Commission:    {backtester.commission:.2%}")
print(f"   Slippage:      {backtester.slippage:.2%}")
print("=" * 60)

🚀 Initializing VectorBT Backtester...
✅ VectorBT Backtester initialized!
   Initial Cash:  $100,000.00
   Commission:    0.10%
   Slippage:      0.05%


## 🔮 Generate Trading Signals from ML Model

In [6]:
# Prepare level files (M, W, D, 1h for multi-timeframe analysis)
level_files = {tf: path for tf, path in test_files.items() if tf in ['M', 'W', 'D', '1h']}

print("🔮 Generating ML trading signals...")
print("=" * 60)

# Generate signals using VectorBT backtester
signals_df = backtester.generate_signals(
    data=test_data_indexed,
    level_files=level_files,
    buy_threshold=BUY_THRESHOLD,
    sell_threshold=SELL_THRESHOLD
)

print("\n📊 Signal Statistics:")
print(f"   Total Candles:  {len(signals_df)}")
print(f"   BUY Signals:    {(signals_df['signal'] == 'BUY').sum()}")
print(f"   SELL Signals:   {(signals_df['signal'] == 'SELL').sum()}")
print(f"   HOLD Signals:   {(signals_df['signal'] == 'HOLD').sum()}")

# Show first few signals
print("\n📋 Sample Signals:")
print(signals_df[['close', 'signal', 'confidence', 'buy_prob', 'sell_prob', 'entries', 'exits']].head(10))

print("\n✅ Signal generation complete!")
print("=" * 60)

🔮 Generating ML trading signals...
🎯 Generating signals for 1000 candles...
📊 Loading levels from: ['1h', 'D', 'W', 'M']
   ⏳ This may take 30-60 seconds (loading historical data)...
🔄 Updating multi-timeframe levels...
Volume Profile: Generated 867 volume profile ranges
Volume Profile: Generated 867 volume profile ranges
✅ Extracted 953 levels from D timeframe
  Period 2020-03: SKIPPED - insufficient data
Volume Profile: Generated 198 volume profile ranges
✅ Extracted 238 levels from W timeframe
Volume Profile: Generated 18 volume profile ranges
✅ Extracted 35 levels from M timeframe
✅ Updated 1226 levels across 3 timeframes
✅ Loaded 1226 support/resistance levels in 2.1s
   Processing 1000 candles...
✅ Extracted 953 levels from D timeframe
  Period 2020-03: SKIPPED - insufficient data
Volume Profile: Generated 198 volume profile ranges
✅ Extracted 238 levels from W timeframe
Volume Profile: Generated 18 volume profile ranges
✅ Extracted 35 levels from M timeframe
✅ Updated 1226 level

## 🚀 Run VectorBT Backtest

In [7]:
# Run the backtest
portfolio = backtester.run_backtest(freq=FREQ)

print("\n🎉 Backtest execution complete!")
print("=" * 60)


🚀 Running VectorBT backtest...
   Initial Capital: $100,000.00
   Commission: 0.10%
   Slippage: 0.05%
✅ Backtest complete!

🎉 Backtest execution complete!
✅ Backtest complete!

🎉 Backtest execution complete!


## 📊 Performance Summary

In [8]:
# Display formatted performance summary
backtester.print_performance_summary()


📊 VECTORBT BACKTEST PERFORMANCE SUMMARY

💰 Returns:
   Total Return:            0.00%
   Annualized Return:       0.00%
   Final Value:       $100,000.00

📈 Risk Metrics:
   Sharpe Ratio:             inf
   Max Drawdown:             nan%

🎯 Trading Performance:
   Total Trades:               0
   Win Rate:                 nan%
   Profit Factor:            nan


💰 Returns:
   Total Return:            0.00%
   Annualized Return:       0.00%
   Final Value:       $100,000.00

📈 Risk Metrics:
   Sharpe Ratio:             inf
   Max Drawdown:             nan%

🎯 Trading Performance:
   Total Trades:               0
   Win Rate:                 nan%
   Profit Factor:            nan



## 📈 Interactive Visualizations

## 🔧 Fix: Install anywidget for Interactive Plots

In [9]:
# Install anywidget in the notebook kernel
import sys
import subprocess

print("📦 Installing anywidget...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "anywidget"])
print("✅ anywidget installed!")
print("\n⚠️  IMPORTANT: Restart the kernel after installation!")
print("   Go to: Kernel → Restart Kernel")

📦 Installing anywidget...
✅ anywidget installed!

⚠️  IMPORTANT: Restart the kernel after installation!
   Go to: Kernel → Restart Kernel


In [10]:
# Generate interactive VectorBT plots
# Using static plots (no anywidget dependency)
backtester.plot_results(use_widgets=False)

# If you want to use widgets after restarting kernel:
# backtester.plot_results(use_widgets=True)


📊 Generating interactive plots...


✅ Plots displayed!


## 📋 Trade-by-Trade Analysis

In [11]:
# Get detailed trade records
trades_df = backtester.get_trade_analysis()

print(f"📋 Total Trades: {len(trades_df)}")
print("\n🔍 Trade Details:")
print(trades_df.head(20))  # Show first 20 trades

# Calculate some additional statistics
if len(trades_df) > 0:
    winning_trades = trades_df[trades_df['PnL'] > 0]
    losing_trades = trades_df[trades_df['PnL'] < 0]
    
    print(f"\n✅ Winning Trades: {len(winning_trades)} ({len(winning_trades)/len(trades_df)*100:.1f}%)")
    print(f"❌ Losing Trades:  {len(losing_trades)} ({len(losing_trades)/len(trades_df)*100:.1f}%)")
    
    if len(winning_trades) > 0:
        print(f"💚 Avg Win:  ${winning_trades['PnL'].mean():.2f}")
        print(f"💚 Max Win:  ${winning_trades['PnL'].max():.2f}")
    
    if len(losing_trades) > 0:
        print(f"💔 Avg Loss: ${losing_trades['PnL'].mean():.2f}")
        print(f"💔 Max Loss: ${losing_trades['PnL'].min():.2f}")
else:
    print("⚠️  No trades executed during backtest period")

📋 Total Trades: 0

🔍 Trade Details:
Empty DataFrame
Columns: [Exit Trade Id, Column, Size, Entry Timestamp, Avg Entry Price, Entry Fees, Exit Timestamp, Avg Exit Price, Exit Fees, PnL, Return, Direction, Status, Position Id]
Index: []
⚠️  No trades executed during backtest period


## 💾 Export Results

In [12]:
# Export all backtest results to files
backtester.export_results(output_dir="backtest_results")

print("\n✅ All results exported successfully!")
print("   Check the 'backtest_results' folder for:")

✅ Saved signals to: backtest_results\signals.csv
✅ Saved trades to: backtest_results\trades.csv
✅ Saved performance stats to: backtest_results\performance_stats.txt
✅ Saved equity curve to: backtest_results\equity_curve.csv

📁 All results exported to: d:\Dev\trading-bot\backtest_results

✅ All results exported successfully!
   Check the 'backtest_results' folder for:
